# Exploration

General exploration to start

In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
pd.set_option('display.max_columns', None)

In [ ]:
raw_data = pd.read_csv('vehicles.csv')
raw_data.columns

In [ ]:
raw_data.describe()

Going to start by looking at the categorical variables to see which ones I can potentially use

In [ ]:
raw_data['condition'].value_counts(dropna=False)  

In [ ]:
raw_data['size'].value_counts(dropna=False)  

In [ ]:
raw_data['type'].value_counts(dropna=False)  

In [ ]:
raw_data['fuel'].value_counts(dropna=False)  

In [ ]:
raw_data['title_status'].value_counts(dropna=False)  

In [ ]:
raw_data['transmission'].value_counts(dropna=False)  

In [ ]:
# Creating histogram of price
# Clearly a right-skewed distribution
raw_data[raw_data['price'] < 100000]['price'].hist(bins=50, edgecolor='black')

In [ ]:
temp = raw_data[(raw_data['price'] < 100000) & (raw_data['odometer'] < 200000)]
plt.figure(figsize=(10, 6))
plt.scatter(temp['odometer'], temp['price'], alpha=0.2)

# Add regression line
slope, intercept, r_value, p_value, std_err = stats.linregress(temp['odometer'], temp['price'])
line = slope * temp['odometer'] + intercept
plt.plot(temp['odometer'], line, 'r-', label=f'R² = {r_value**2:.3f}')

plt.xlabel('Mileage')
plt.ylabel('Car Price')
plt.title('Car Price vs Mileage')
plt.legend()
plt.show()

# Filtering

Based on the exploration above, the columns that seem like the best to keep are the following. I may end up dropping more columns later if they don't add value

In [ ]:
columns = ['price','year','manufacturer','model','condition','cylinders','fuel','odometer','title_status','transmission','drive','size','type','paint_color','state']

In [ ]:
df = raw_data[columns]

In [ ]:
total_rows = len(df)
total_rows_no_na = len(df.dropna())
percentage_rows_dropped = (total_rows - total_rows_no_na) / total_rows * 100

print(f'Percentage of rows dropped: {percentage_rows_dropped:.2f}%')
print(f'This leaves us with {total_rows_no_na} rows out of {total_rows}.')

While we are dropping a lot of rows, since these rows are quality, we still have more than enough data to make a good model

In [ ]:
df = df.dropna()

In [ ]:
# Keeping only manufacturers with more than 1000 cars. Don't wan to handle rare cases.
manufacturers = df['manufacturer'].value_counts()
manufacturers = manufacturers[manufacturers > 1000]
df = df[df['manufacturer'].isin(manufacturers.index)]

In [ ]:
def normalize_text(text):
    if pd.isnull(text):
        return ""
    text = text.lower()
    text = re.sub(r'[^a-z0-9 ]', '', text) 
    text = re.sub(r'\s+', ' ', text).strip() 
    return text
df['model_clean'] = df['model'].apply(normalize_text)

In [ ]:
model_counts = df['model_clean'].value_counts()
common_models = model_counts[model_counts >= 100].index
df = df[df['model_clean'].isin(common_models)].copy()

# Combining Make and Model

Note, I originally tried modeling based off of "size" and "type" and "manufacturer", so "Ford Mid-Sized SUV". While this actually worked pretty well, that is not how people search for cars. They search for makes/models. So even though it wasn't perfect, I decided to clean up the make and model and just drop the makes and models that weren't very common

In [ ]:
def clean_model(text):
    if pd.isnull(text):
        return ""
    text = text.lower()
    text = re.sub(r'[^a-z0-9 ]', '', text) 
    text = re.sub(r'\s+', ' ', text).strip() 
    return text

df['cleaned_model'] = df['model'].apply(clean_model)

In [ ]:
models = [
    # Ford models
    'f150',
    'escape',
    'explorer',
    'f250',
    'mustang',
    'fusion',
    'focus',
    'f350',
    'edge',
    'expedition',
    # Chevrolet models
    'silverado',
    'equinox',
    'malibu',
    'tahoe',
    'corvette',
    'impala',
    'cruze',
    'camaro',
    'suburban',
    'traverse',
    # Toyota models
    'camry',
    'tacoma',
    'corolla',
    'rav4',
    'prius',
    'tundra',
    '4runner',
    'sienna',
    'highlander',
    # Honda models
    'accord',
    'civic',
    'crv',
    'odyssey',
    'pilot',
    # Nissan models
    'altima',
    'rogue',
    'sentra',
    'maxima',
    'pathfinder',
    'versa',
    'murano',
    'frontier',
    # Jeep models
    'wrangler',
    'grand cherokee', # Important this is before cherokee for matching
    'cherokee',
    'liberty',
    'patriot',
    'compass',
    # Ram models
    '1500',
    '2500',
    '3500',
    # GMC models
    'sierra',
    'acadia',
    'yukon',
    'terrain',
    # Note: Left BMW off since naming conditions are too varied
    # Dodge models
    'charger',
    'grand caravan',
    'challenger',
    'durango',
    'journey',
    # Mercedes-Benz models
    'cclass',
    'eclass',
    'sclass',
    # Hyundai models
    'sonata',
    'elantra',
    'santa fe',
    'tucson',
    # Subaru models
    'outback',
    'forester',
    'impreza',
    'legacy',
    # Volkswagen models
    'jetta',
    'passat',
    'tiguan',
    # Kia models
    'soul',
    'optima',
    'sorento',
    'forte', 
]

In [ ]:
df['base_model'] = 'other' 
for model in models:
    mask = df['cleaned_model'].str.contains(model, case=False, na=False)
    df.loc[mask, 'base_model'] = model

# Checking for Outliers

In [ ]:
# These cars over 1 million dollars are clearly mislabeled
df[df['price'] >1000000].head(3)

In [ ]:
df['year'].describe()

In [ ]:
df['odometer'].describe()

Clearly some cars were mislabeled with price. I will be using 2.5 times the IQR to detect outliers for price and odometer. If this range goes below 0, may need to filter out 0 manually since this wouldn't make since for either column. For year I will be setting a cutoff manually since I know vintage cars can do the inverse in what we normally see in price and that is not what this model is going to be focused on